# IFC Export using Autodesk Automation API

This notebook demonstrates how to export IFC files from Revit models using the Autodesk Automation API. The process is broken down into clear steps.

## Step 1: Setup and Import Libraries

Import all necessary libraries and modules.

In [1]:
import os
import json
import logging
from pathlib import Path
from dotenv import load_dotenv

from aps_automation_sdk.classes import (
    Activity,
    ActivityInputParameter,
    ActivityOutputParameter,
    ActivityJsonParameter,
    AppBundle,
    WorkItem
)

from aps_automation_sdk.utils import (
    delete_activity,
    delete_appbundle,
    get_token,
    get_nickname
)

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")

## Step 2: Load Environment Variables and Configure Authentication

Load credentials from environment variables and get an authentication token.

In [2]:
load_dotenv()

CLIENT_ID = os.getenv("CLIENT_ID", "")
CLIENT_SECRET = os.getenv("CLIENT_SECRET", "")

# Get authentication token and nickname
token = get_token(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
nickname = get_nickname(token)

print(f"Authentication successful. Nickname: {nickname}")

Authentication successful. Nickname: viktortest


## Step 3: Define Project Constants

Set up all the constants needed for the IFC export workflow.

In [3]:
# Define constants
app_bundle_name = "IfcExportBundle"
activity_name = "RevitIfcExportAppActivity"
alias = "prod"
bucket_key = "vkt001_bucket_ifc_export"
zip_path = Path.cwd() / "bundles" / "export-ifc-addin" / "IfcExportDA.bundle.zip"

# Create full aliases
appbundle_full_alias = f"{nickname}.{app_bundle_name}+{alias}"
activity_full_alias = f"{nickname}.{activity_name}+{alias}"

print(f"App Bundle: {appbundle_full_alias}")
print(f"Activity: {activity_full_alias}")

App Bundle: viktortest.IfcExportBundle+prod
Activity: viktortest.RevitIfcExportAppActivity+prod


## Step 4: Create and Deploy App Bundle

Register and upload the app bundle that contains the IFC export logic.

In [4]:
bundle = AppBundle(
    appBundleId=app_bundle_name,
    engine="Autodesk.Revit+2024",
    alias=alias,
    zip_path=str(zip_path),
    description="Exports IFC from selected views",
)

bundle.deploy(token)
print("App bundle deployed successfully!")

App bundle deployed successfully!


## Step 5: Define Activity Parameters

Create input and output parameters for the activity.

In [6]:
# Input: Revit file
input_revit = ActivityInputParameter(
    name="rvtFile",
    localName="input.rvt",
    verb="get",
    description="Input Revit File",
    required=True,
    is_engine_input=True,
    bucketKey=bucket_key,
    objectKey="input.rvt",
)

# Output: Zipped IFC files
output_zip = ActivityOutputParameter(
    name="result",
    localName="result",
    verb="put",
    description="Zipped IFCs",
    zip=True,
    bucketKey=bucket_key,
    objectKey="IFCExport.zip",
)

# Input: JSON settings
input_json = ActivityJsonParameter(
    name="ifcSettings",
    localName="ifc_settings.json",
    verb="get",
    description="Export Settings Parameter JSON",
)

print("Activity parameters defined successfully!")

Activity parameters defined successfully!


## Step 6: Create and Deploy Activity

Create the activity that links the app bundle with the parameters.

In [ ]:
activity = Activity(
    id=activity_name,
    parameters=[input_revit, output_zip, input_json],
    engine="Autodesk.Revit+2024",
    appbundle_full_name=appbundle_full_alias,
    description="Export IFC from views using JSON settings",
    alias=alias,
)

# Set the Revit command
activity.set_revit_command_line()
activity.deploy(token=token)
print("Activity deployed successfully!")

Activity deployed successfully!


## Step 7: Upload Input Revit File

Upload the Revit file to Object Storage Service (OSS).

In [8]:
input_rvt_path = Path.cwd() / "bundles" / "export-ifc-addin" / "SteelStructureFrame.rvt"
input_revit.upload_file_to_oss(file_path=str(input_rvt_path), token=token)
print(f"Input Revit file uploaded: {input_rvt_path.name}")

Input Revit file uploaded: SteelStructureFrame.rvt


## Step 8: Load and Set IFC Export Settings

Read the IFC export settings from JSON and set them for the work item.

In [9]:
settings_path = Path.cwd() / "bundles" / "export-ifc-addin" / "export_settings.json"
with settings_path.open("r", encoding="utf-8") as f:
    settings = json.load(f)

input_json.set_content(settings)
print(f"IFC settings loaded from: {settings_path.name}")
print(f"Settings: {json.dumps(settings, indent=2)}")

IFC settings loaded from: export_settings.json
Settings: {
  "view_names": [
    "Foundations"
  ],
  "FileVersion": "IFC4",
  "IFCFileType": "IFC",
  "ExportBaseQuantities": true,
  "SpaceBoundaryLevel": 2,
  "FamilyMappingFile": "C:\\Data\\ifc_mapping.txt",
  "ExportInternalRevitPropertySets": false,
  "ExportIFCCommonPropertySets": true,
  "ExportAnnotations": false,
  "Export2DElements": false,
  "ExportRoomsInView": false,
  "VisibleElementsOfCurrentView": false,
  "ExportLinkedFiles": false,
  "IncludeSteelElements": false,
  "ExportPartsAsBuildingElements": true,
  "UseActiveViewGeometry": false,
  "UseFamilyAndTypeNameForReference": false,
  "Use2DRoomBoundaryForVolume": false,
  "IncludeSiteElevation": false,
  "ExportBoundingBox": false,
  "ExportSolidModelRep": false,
  "StoreIFCGUID": false,
  "ExportSchedulesAsPsets": false,
  "ExportSpecificSchedules": false,
  "ExportUserDefinedPsets": false,
  "ExportUserDefinedPsetsFileName": "",
  "ExportUserDefinedParameterMapping": 

## Step 9: Create and Execute Work Item

Create a work item with all parameters and execute it.

In [10]:
work_item = WorkItem(
    parameters=[input_revit, output_zip, input_json],
    activity_full_alias=activity_full_alias
)

print("Work item created. Starting execution...")

Work item created. Starting execution...


## Step 10: Poll Work Item Status

Monitor the work item execution until completion (max 10 minutes).

In [11]:
status_resp = work_item.execute(token=token, max_wait=600, interval=10)
last_status = status_resp.get("status", "")

print(f"Work item completed with status: {last_status}")
print(f"Full response: {json.dumps(status_resp, indent=2)}")

2025-10-29 22:14:02,770 INFO Polling work item status, id=8db4584fd14d4768891e04d14709565a
2025-10-29 22:14:03,774 INFO [  0s] status=inprogress report_url=None
2025-10-29 22:14:14,754 INFO [ 10s] status=inprogress report_url=None
2025-10-29 22:14:25,733 INFO [ 20s] status=inprogress report_url=None
2025-10-29 22:14:36,750 INFO [ 30s] status=inprogress report_url=None
2025-10-29 22:14:49,233 INFO [ 40s] status=inprogress report_url=None
2025-10-29 22:15:00,182 INFO [ 50s] status=inprogress report_url=None
2025-10-29 22:15:11,174 INFO [ 60s] status=inprogress report_url=None
2025-10-29 22:15:22,174 INFO [ 70s] status=success report_url=https://dasprod-store.s3.amazonaws.com/workItem/viktortest/8db4584fd14d4768891e04d14709565a/report?X-Amz-Expires=3600&X-Amz-Security-Token=IQoJb3JpZ2luX2VjECsaCXVzLWVhc3QtMSJIMEYCIQD2bmVs%2F6bqrDrftrkShWpTkmzY95W%2FXxCed6miMDgDfQIhALCX5FBQccW08V1G8F9LzqtwJy6pUaA2r5MoeWVrhEFEKokDCOP%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEQBBoMMjIwNDczMTUyMzEwIgyaTSQr8GoG8JgcFfsq3Q

Work item completed with status: success
Full response: {
  "status": "success",
  "reportUrl": "https://dasprod-store.s3.amazonaws.com/workItem/viktortest/8db4584fd14d4768891e04d14709565a/report?X-Amz-Expires=3600&X-Amz-Security-Token=IQoJb3JpZ2luX2VjECsaCXVzLWVhc3QtMSJIMEYCIQD2bmVs%2F6bqrDrftrkShWpTkmzY95W%2FXxCed6miMDgDfQIhALCX5FBQccW08V1G8F9LzqtwJy6pUaA2r5MoeWVrhEFEKokDCOP%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEQBBoMMjIwNDczMTUyMzEwIgyaTSQr8GoG8JgcFfsq3QI%2BK5qQdpd4aWOYTGfbVhGj22af8EszYpblm4gp%2FWnbMMDUwButpDgjUicikhR%2Fj8p5F0A31PgC4%2BdtlUitXa4in%2FqLWi3VNvWQVTHxjXkdp4r2jwSjxBsy30gCvXkIr9jPnJwvJB0LxUhlb0Sz9OWVbGpHpLLrLZAsoXiMAKoEzUUsZEmTb6YkRLGNPp%2FYFYogcCeiWQfFA9oESE44aI19YTF8GA99Ygupre8GoCuoPYRSlETDZHwpInM1jfzv2x%2Fnb23aL4Lb5pjJ9ClKOtkU6UwXaQ3GTwAyzVZqCBzegMFyAOG5drzGWYJJb2DZYAkWI12fY9FFtIbeKKnmOEZbxdiOqHqZVDcTU7U0uIP9KMYjvQfQkAMMmg3J6aN%2Bn0o8w66B35fgm3myQ28Hd6ABes1Ihl9dIW99qC%2F7w2Ymyslll95fNiGVtSzergF11g2544LYE68tq2x2lygIMNCPi8gGOp0BY7beWH%2F%2F9tbsElEwbSgEf6m2HMWaQcvlrF%2FjW3%2Fbh%

## Step 11: Download Results

If successful, download the IFC export results.

In [12]:
if last_status == "success":
    out_dir = Path.cwd() / "output"
    out_dir.mkdir(parents=True, exist_ok=True)
    out_zip = out_dir / "IFCExport.zip"
    output_zip.download_to(output_path=str(out_zip), token=token)
    print(f"✅ Download successful: {out_zip}")
else:
    print(f"❌ Work item failed or did not complete successfully. Status: {last_status}")

✅ Download successful: c:\Users\aleja\Documents\aps_automation_sdk\output\IFCExport.zip


## Step 12: Cleanup Resources

Delete the activity and app bundle to clean up resources.

In [13]:
print("Cleaning up: deleting activity and app bundle...")

try:
    delete_activity(activityId=activity_name, token=token)
    print(f"✅ Activity deleted: {activity_full_alias}")
except Exception as e:
    print(f"⚠️ Failed to delete activity: {e}")

try:
    delete_appbundle(appbundleId=app_bundle_name, token=token)
    print(f"✅ App bundle deleted: {app_bundle_name}")
except Exception as e:
    print(f"⚠️ Failed to delete app bundle: {e}")

print("\n🎉 IFC Export workflow completed!")

Cleaning up: deleting activity and app bundle...
✅ Activity deleted: viktortest.RevitIfcExportAppActivity+prod
✅ App bundle deleted: IfcExportBundle

🎉 IFC Export workflow completed!
